In [1]:
import pandas as pd

# Load the CSV file to examine its contents
csv_path = 'MontpellierFrequenceHourlyPT.csv'
df = pd.read_csv(csv_path, sep=',',parse_dates=['date'])

# Display the first few rows of the dataframe to understand its structure
df

,date,index,PT
0,2023-01-02,1,40
1,2023-01-02,2,36
2,2023-01-02,3,44
3,2023-01-02,4,23
4,2023-01-02,5,15
...,...,...,...
2250,2023-12-29,5,12
2251,2023-12-29,6,17
2252,2023-12-29,7,41
2253,2023-12-29,8,33


In [2]:
data = df.copy()
# Convert 'date' to datetime and extract the day of the week
data['day_of_week'] = data['date'].dt.day_name()

# Drop the original date column as we now have the day of the week
data = data.drop('date', axis=1)

# Group the data by 'day_of_week' and 'index' and calculate the mean frequency
weekly_avg = data.groupby(['day_of_week', 'index']).mean().reset_index()
weekly_avg.head()

,day_of_week,index,PT
0,Friday,1,32.66
1,Friday,2,35.70
2,Friday,3,38.28
3,Friday,4,26.88
4,Friday,5,24.44


In [4]:

# Pivot the table to have 'index' as rows and 'day_of_week' as columns with the mean frequency as values
pivot_table = weekly_avg.pivot(index='index', columns='day_of_week', values='PT').reset_index()

# Re-order the days to match typical French week starting from Monday
ordered_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
pivot_table = pivot_table[["index"] + ordered_days]

# Determine the thresholds for the colors in the original image
# Since we don't have the exact values, we can infer them based on quantiles
# Green: Low (0-25%), Yellow: Medium (25-75%), Red: High (75-100%)

thresholds = {
    'green': pivot_table.quantile(0.25),
    'yellow': pivot_table.quantile(0.75),
    'red': pivot_table.max(),  # This assumes the max value is considered high
}

pivot_table

day_of_week,index,Monday,Tuesday,Wednesday,Thursday,Friday
0,1,46.340426,31.760000,30.480000,16.100000,32.66
1,2,41.000000,34.294118,33.160000,31.431373,35.70
2,3,43.893617,35.392157,38.320000,37.705882,38.28
3,4,29.893617,26.784314,27.300000,26.823529,26.88
4,5,24.872340,22.392157,24.588235,19.117647,24.44
5,6,33.276596,28.607843,30.529412,31.686275,30.06
6,7,41.063830,37.235294,41.098039,40.411765,39.74
7,8,38.765957,35.294118,36.568627,37.196078,36.56
8,9,14.510638,13.549020,16.627451,24.823529,15.84
9,10,NaN,NaN,NaN,23.176471,NaN


In [7]:
color_coded_df = pivot_table.copy()

# Color code the values based on the thresholds
for day in ordered_days:
    color_coded_df[day] = pd.cut(
        pivot_table[day],
        bins=[-float("inf"), thresholds['green'][day], thresholds['yellow'][day], thresholds['red'][day], float("inf")],
        labels=['green', 'yellow', 'red', 'black']
    )
# replace Nan with black
color_coded_df = color_coded_df.fillna('black')

color_coded_df

day_of_week,index,Monday,Tuesday,Wednesday,Thursday,Friday
0,1,red,yellow,yellow,green,yellow
1,2,yellow,yellow,yellow,yellow,yellow
2,3,red,red,red,red,red
3,4,green,green,green,yellow,green
4,5,green,green,green,green,green
5,6,yellow,yellow,yellow,yellow,yellow
6,7,yellow,red,red,red,red
7,8,yellow,yellow,yellow,red,yellow
8,9,green,green,green,yellow,green
9,10,black,black,black,green,black


In [71]:
# Define the time slots from the image
time_slots = [
    "08:30 à 09:30",
    "09:30 à 10:30",
    "10:30 à 11:30",
    "11:30 à 12:30",
    "12:30 à 13:30",
    "13:30 à 14:30",
    "14:30 à 15:30",
    "15:30 à 16:30",
    "16:30 à 17:30",
    "17:30 à 19:00"
]

# Drop the 'index' column
color_coded_df.drop('index', axis=1, inplace=True)

# Set the new index to the time slots
color_coded_df.index = time_slots
color_coded_df

day_of_week,Monday,Tuesday,Wednesday,Thursday,Friday
08:30 à 09:30,yellow,yellow,yellow,green,yellow
09:30 à 10:30,red,yellow,yellow,green,yellow
10:30 à 11:30,red,red,red,red,red
11:30 à 12:30,green,yellow,green,yellow,green
12:30 à 13:30,green,green,green,green,green
13:30 à 14:30,yellow,yellow,yellow,yellow,yellow
14:30 à 15:30,yellow,red,yellow,red,yellow
15:30 à 16:30,yellow,red,red,red,red
16:30 à 17:30,green,green,green,yellow,green
17:30 à 19:00,black,green,black,yellow,black
